# init

In [1]:
import pygame
import numpy as np
import numpy.linalg as lin
import sys
import importlib
import itertools
import time

import Clipping
from Geometry import *
from camera import Camera
from draw import *

# reload modules

In [47]:

importlib.reload(Clipping)
importlib.reload(sys.modules['Geometry'])
importlib.reload(sys.modules['vec'])
from Geometry import *

importlib.reload(sys.modules['camera'])
from camera import Camera

importlib.reload(sys.modules['draw'])
from draw import *




# some out of date crap still being used

In [10]:
def vert_list_to_lines(verts,cycle=True):
    cycle_edges = np.stack((verts,np.roll(verts,1,axis=0))).transpose([1,0,2])
    if cycle and len(verts) > 2:
        return cycle_edges
    else:
        return cycle_edges[:-1]
    
square = np.array([[1,1,0],[-1,1,0],[-1,-1,0],[1,-1,0]])
square1 = square + np.array([0,0,-1])
square2 = square + np.array([0,0,1])
connect = np.array([[[1,1,-1],[1,1,1]],[[-1,1,1],[-1,1,-1]],[[-1,-1,-1],[-1,-1,1]],[[1,-1,-1],[1,-1,1]]])
cube_lines_test = 2*np.concatenate((np.concatenate(list(map(vert_list_to_lines,[square1,square2]))),connect))

# build shapes

In [35]:
from colors import *
    
d = 4 #set dimensionality
cube_face_colors = [RED,YELLOW,GREEN,CYAN,MAGENTA, WHITE]
tess_face_colors = cube_face_colors + [ORANGE,PURPLE]

cube_shape = build_cube(d)


if d == 4:
    for face, color in zip(cube_shape.faces,tess_face_colors):
        face.color = color
    shapes = []
    for i in range(4):
        cube = cube_shape.copy()
        cube.update(pos = 2*np.eye(d)[i])
        shapes.append(cube)
        cube = cube_shape.copy()
        cube.update(pos = -2*np.eye(d)[i])
        shapes.append(cube)
    
    #shapes = [cube_shape] #single cube
    #cube_shape.transparent = True
if d == 3:
    for face, color in zip(cube_shape.faces,cube_face_colors):
        face.color = color
    cube1 = cube_shape.copy()
    cube1.update(pos=np.array([2,0,0]))

    cube2 = cube_shape.copy()
    cube2.update(pos=np.array([-2,0,0]))

    cube3 = cube_shape.copy()
    cube3.update(pos=np.array([0,0,2]))

    cube4 = cube_shape.copy()
    cube4.update(pos=np.array([0,0,-2]))

    cube5 = cube_shape.copy()
    cube5.update(pos=np.array([0,-2,0]))
    
    shapes = [cube1,cube2,cube3,cube4,cube5]
for i,shape in enumerate(shapes):
    shape.name = 'shape ' + str(i)

shape_combinations = list(itertools.combinations(shapes,2))

# main loop

In [51]:

t = 0
#size = width, height = 1100, 700
size = width, height = 1800, 900

pygame.init()

if d == 3:
    draw_class = DrawOpenGL2D(pygame,size,draw_origin=np.zeros([d-1]))
    camera = Camera(pos = np.array([0.8,2,0.8]),draw_class = draw_class)
    camera.update_rot_matrix(0,-1,-3*np.pi/4)
    camera.update_rot_matrix(1,-1,-np.pi/4)
if d == 4:
    draw_class = DrawOpenGL3D(pygame,size,
                              draw_origin=np.array([0.0,0.0,-15.0]),
                              focal=4,stereo=False)
    if draw_class.stereo:
        draw_class.view_radius = 4
    else:
        draw_class.view_radius = 6
    #draw_class.stereo_view_angles = [[-30,0],[0,0]]
    camera = Camera(pos = np.array([0,0,0,0]),draw_class = draw_class)
    #camera.look_at(np.array([2,2,2,2]))
    #camera = Camera(pos = np.array([0.8,2,0.8,0.8]),draw_class = draw_class)
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(2,-1,-np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
#camera.update_rot_matrix(2,3,-np.pi/4)
camera.clipping = True
quit = False
redraw = True
timer = time.time()
frame = 0
while not quit:
    
    if redraw:
        draw_class.draw(camera,shapes)
#         if d == 4:
#             draw_class.draw_lines_3d(cube_lines_test,GREEN,draw_origin = draw_class.draw_origin + )
#             draw_class.draw_lines_3d(cube_lines_test,GREEN)
        pygame.display.flip()
        redraw = False
    
    events = pygame.event.get()
    
    redraw = camera.update(draw_class,events) #this is here so that we draw on the first frame
    redraw = True #redraw every loop. useful for assessing framerate
    for event in events:
        if event.type == pygame.QUIT:
            quit = True
        #toggle transparency
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_t:
                for shape in shapes:
                    shape.transparent = not shape.transparent
                    redraw = True
                print('transparency',shapes[0].transparent)
    keys = pygame.key.get_pressed()
    if keys[pygame.K_ESCAPE]:
        quit = True
    
    delta_t = (time.time() - timer)
    if delta_t > 3:
    #if False:
        #print('redraw:',redraw)
        try:
            #fps = 30/(time.time() - timer)
            fps = frame/delta_t
        except:
            fps = 0
        #if fps < 60:
        print(fps,'fps')
        timer = time.time()
        frame = 0
    frame = frame+1
pygame.quit()
print('game terminated')

40.02252001086549 fps
21.31188918519229 fps
29.31228834685282 fps
26.498492825819312 fps
20.42359288535656 fps
transparency False
18.113239310237663 fps
11.924209712333418 fps
11.925077991436876 fps
game terminated


In [46]:
pygame.quit()

In [17]:
[cube.verts[vi] for vi in cube.edges[3]]

[array([-1., -1., -1., -3.]), array([ 0., -1., -1., -3.])]

In [263]:
#test minor extraction
verts = np.array([[1,0,0],[0,1,0],[1,1,0]])
dvs = verts - np.roll(verts,1,axis=0)
print(dvs[:-1])
dvs[:2][:,[0,2]]

[[ 0 -1  0]
 [-1  1  0]]


array([[ 0,  0],
       [-1,  0]])

In [24]:
i

888362